In [ ]:
import voeventcache

In [ ]:
import logging
logging.basicConfig()

In [ ]:
from voeventcache.tests.resources import swift_bat_grb_pos_v2_etree
from voeventcache.database.models import Voevent, Base
row = Voevent.from_etree(swift_bat_grb_pos_v2_etree)

In [ ]:
from voeventcache.database import db_utils
from voeventcache.tests.config import testdb_scratch_url, admin_db_url
if not db_utils.check_database_exists(testdb_scratch_url):
    db_utils.create_database(admin_db_url, testdb_scratch_url.database)
    engine = sqlalchemy.engine.create_engine(testdb_scratch_url)
    Base.metadata.create_all(engine)
    engine.dispose()

In [ ]:
from sqlalchemy.orm import sessionmaker
engine = sqlalchemy.engine.create_engine(testdb_manual_url)
Session = sessionmaker(bind=engine)
s = Session()
s.query(Voevent).first()
print

In [ ]:
print s.query(Voevent.ivorn, Voevent.received).all()[0]

In [ ]:
v0 = s.query(Voevent).all()[0]
# s.add(row)
# s.commit()

In [ ]:
v0